In [3]:
LOG_PATH = "../log"

# Fetch Inervals from log file

In [4]:
import pandas as pd
import time
import datetime

def getIntervalTemplate(outputFile):
    # Get intermediate file
    import subprocess
    bashCommand = '''echo "time" > interval_intermediate.csv && cat '''+LOG_PATH+'''/keyStroke.log | grep "\-\-\-5min" | grep -o '[0-9]*-[0-9]*-[0-9]*-[0-9]*:[0-9]*:[0-9]*' >> ./interval_intermediate.csv'''
    subprocess.check_output(bashCommand, shell=True)


    raw_time = pd.read_csv("interval_intermediate.csv")
    raw_time = pd.DataFrame(raw_time)

    all_time_stamp = []
    dateTime = str(raw_time.time[0])
    unixTime = time.mktime(datetime.datetime.strptime(dateTime, "%Y-%m-%d-%H:%M:%S").timetuple())
    for i in range(1,len(raw_time.time)):
        new_unixTime = time.mktime(datetime.datetime.strptime(str(raw_time.time[i]), "%Y-%m-%d-%H:%M:%S").timetuple())      
        all_time_stamp.append(datetime.datetime.fromtimestamp(unixTime).strftime('%Y-%m-%d-%H:%M:%S')+"--"+datetime.datetime.fromtimestamp(new_unixTime).strftime('%Y-%m-%d-%H:%M:%S'))
        unixTime = new_unixTime
    df = pd.DataFrame(all_time_stamp, columns=['time_intervals'])
    df.to_csv(outputFile, index=False, sep=' ')
    return df
    
getIntervalTemplate("interval_template.csv")[:5]

,time_intervals
0,2018-12-10-11:58:31--2018-12-10-11:59:00
1,2018-12-10-11:59:00--2018-12-10-12:05:04


# Fetch number of windows being touched in 5 mins

In [5]:
def windowTouchesCount():
    count = 0
    start = True
    windowContent = [line.strip() for line in open("./"+LOG_PATH+"/windowAudit.log", 'r')]
    windowCount_feature = []
    for line in windowContent:
        if line != "":
            if "---5min Window Audit milestone" in line:
                if start:
                    start = False
                else:
                    windowCount_feature.append(count)
                    count = 0 
            else:
                count += 1
    df = pd.DataFrame(windowCount_feature, columns=['touched_windows_count'])
    df.to_csv("touched_windows_count.csv", index=False, sep=' ')
    return df

windowTouchesCount()[:5]

,touched_windows_count
0,0
1,3


# Fetch Chrome, WeChat, Terminal lasting time in 5 mins (write in class)

In [11]:
class Interval(object):
    def __init__(self, start, end):
        self.start = start
        self.end = end 
        self.lasting = 0

def unixTime_intervals(element):
    fromTime, toTime = element.split("--")
    # Conver to unix time
    fromTime = time.mktime(datetime.datetime.strptime(fromTime, "%Y-%m-%d-%H:%M:%S").timetuple()) 
    toTime = time.mktime(datetime.datetime.strptime(toTime, "%Y-%m-%d-%H:%M:%S").timetuple()) 
    return fromTime, toTime

def unixTime_windows(element):
    currentStart = element[:31]
    currentEnd = element[32:63]
    # Conver to Unix time
    currentStart = time.mktime(datetime.datetime.strptime(currentStart, "%I:%M:%S%p on %B %d, %Y").timetuple())
    currentEnd = time.mktime(datetime.datetime.strptime(currentEnd, "%I:%M:%S%p on %B %d, %Y").timetuple())
    return currentStart, currentEnd

def buildIntervalObject(dataList):
    intervals = []
    for line in dataList:
        start, end = unixTime_intervals(line)
        intervals.append(Interval(start, end))
    return intervals

def buildWindowObject(dataList):
    intervals = []
    for i in range(len(dataList)):
        if intervals == [] or dataList[i] != dataList[i-1]:
            start, end = unixTime_windows(dataList[i])
            intervals.append(Interval(start, end))
    return intervals

def lastingTimeRatio(appName):
    # Get interval template's time
    raw_intervals = pd.read_csv("interval_template.csv")
    raw_intervals = pd.DataFrame(raw_intervals)
    # package to Interval object
    intervals = buildIntervalObject(raw_intervals.time_intervals)
    
    # clean up windows log
    windowContent = [line.strip() for line in open("./"+LOG_PATH+"/windowAudit.log", 'r')]
    windowContent = [line for line in windowContent if line != "" and "---5min Window Audit milestone" not in line and appName in line]
    # package to Interval object
    appBucket = buildWindowObject(windowContent)


    # Two pointers
    i, j = 0, 0
    appLastingTime_ratio = []
    current = appBucket[0] if appBucket != [] else None
    inv = intervals[0]
    
    while j < len(appBucket) and i < len(intervals):
        inv = intervals[i]
        if current.start > inv.end:
            i += 1
            appLastingTime_ratio.append(inv.lasting * 1.0 / (inv.end - inv.start))
        else:
            if current.end <= inv.end:
                j += 1
                inv.lasting += (current.end - current.start)
                if j < len(appBucket):
                    current = appBucket[j]
            else:
                inv.lasting += (min(inv.end, current.end) - current.start)
                current.start = inv.end
                i += 1
                appLastingTime_ratio.append(inv.lasting * 1.0 / (inv.end - inv.start))

    # Traverse the rest time intervals
    while i < len(intervals):
        inv = intervals[i]
        appLastingTime_ratio.append(inv.lasting * 1.0 / (inv.end - inv.start))
        i += 1
    
    df = pd.DataFrame(appLastingTime_ratio, columns=[appName + '_ratio'])
    df.to_csv(appName + "_ratio.csv", index=False, sep=' ')
    return df



In [12]:
lastingTimeRatio("Google Chrome")[:5]

,Google Chrome_ratio
0,0.000000
1,0.008242


In [13]:
lastingTimeRatio("WeChat")[:5]

,WeChat_ratio
0,0.000000
1,0.065934


In [14]:
lastingTimeRatio("Terminal")[:5]

,Terminal_ratio
0,0.000000
1,0.002747


In [15]:
lastingTimeRatio("Code")[:5]

,Code_ratio
0,0.0
1,0.0


# Fetch Chrome Tab Count in 5 mins

In [16]:
def chromeTabCount():
    chromeTabCountContent = [line.strip() for line in open("./"+LOG_PATH+"/chromeTabCount.log", 'r')]
    chromeTabCountContent = [line for line in chromeTabCountContent if line != "" and "---5min Chrome Tab Count Audit milestone" not in line]
    df = pd.DataFrame(chromeTabCountContent, columns=['chrome_tab_count'])
    df.to_csv("chrome_tab_count.csv", index=False, sep=' ')
    return df

chromeTabCount()[:5]

,chrome_tab_count
0,29
1,29
2,29
3,1


# Fetch Chrome Tab's activity in 5 mins

In [17]:
def chromeTabOpen():
    chromeTabActContent = [line.strip() for line in open("./"+LOG_PATH+"/chromeTab.log", 'r')]
    openAct = []
    start = True
    count = 0
    for line in chromeTabActContent:
        if "---5min Chrome Tab Audit milestone" in line:
            # Ignore the first line
            if start:
                start = False
            else:
                openAct.append(count)
                count = 0
        elif "open" in line:
            count += 1
    openAct.append(count)
        
    df = pd.DataFrame(openAct, columns=['chrome_tab_open_act'])
    df.to_csv("chrome_tab_open_act.csv", index=False, sep=' ')
    return df

chromeTabOpen()[:5]

,chrome_tab_open_act
0,29
1,29
2,29


In [18]:
def chromeTabClose():
    chromeTabActContent = [line.strip() for line in open("./"+LOG_PATH+"/chromeTab.log", 'r')]
    openAct = []
    start = True
    count = 0
    for line in chromeTabActContent:
        if "---5min Chrome Tab Audit milestone" in line:
            # Ignore the first line
            if start:
                start = False
            else:
                openAct.append(count)
                count = 0
        elif "close" in line:
            count += 1
    openAct.append(count)
        
    df = pd.DataFrame(openAct, columns=['chrome_tab_close_act'])
    df.to_csv("chrome_tab_close_act.csv", index=False, sep=' ')
    return df

chromeTabClose()[:5]

,chrome_tab_close_act
0,0
1,0
2,0


# 5 mins filter

In [52]:
import numpy as np

# Concatenate features
def concatenate(fileList, outputFileName):
    import pandas as pd

    dfs = []
    for filename in fileList:
        # read the csv, making sure the first two columns are str
        df = pd.read_csv(filename, header=None, converters={0: str, 1: str})
        # throw away all but the first two columns
        df = df.ix[:,:1]
        # change the column names so they won't collide during concatenation
        df.columns = [filename for cname in df.columns]
        dfs.append(df)
    
    
    # concatenate them horizontally
    merged = pd.concat(dfs,axis=1)
    dropNAN = merged.dropna()
    # write it out
    dropNAN.to_csv(outputFileName, header=None, index=None)
    return dropNAN


concatenate(["interval_template.csv","touched_windows_count.csv", "Google Chrome_ratio.csv", "WeChat_ratio.csv", "Terminal_ratio.csv", "Code_ratio.csv", "chrome_tab_count.csv", "chrome_tab_open_act.csv", "chrome_tab_close_act.csv", "cmdTabCombo_count.csv", "cmd_count.csv", "avg_typing_speed.csv", "port_opened_count.csv", "port_closed_count.csv", "process_opened_count.csv", "process_closed_count.csv", "document_touched_count.csv", "image_touched_count.csv", "decoy_touched_count.csv"], "features_intermediate.csv")[:5]

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if sys.path[0] == '':


,interval_template.csv,touched_windows_count.csv,Google Chrome_ratio.csv,WeChat_ratio.csv,Terminal_ratio.csv,Code_ratio.csv,chrome_tab_count.csv,chrome_tab_open_act.csv,chrome_tab_close_act.csv,cmdTabCombo_count.csv,cmd_count.csv,avg_typing_speed.csv,port_opened_count.csv,port_closed_count.csv,process_opened_count.csv,process_closed_count.csv,document_touched_count.csv,image_touched_count.csv,decoy_touched_count.csv
0,time_intervals,touched_windows_count,Google Chrome_ratio,WeChat_ratio,Terminal_ratio,Code_ratio,chrome_tab_count,chrome_tab_open_act,chrome_tab_close_act,cmdTabCombo_count,cmd_count,avg_typing_speed,port_opened_count,port_closed_count,process_opened_count,process_closed_count,document_touched_count,image_touched_count,decoy_touched_count
1,2018-12-10-11:58:31--2018-12-10-11:59:00,0,0.0,0.0,0.0,0.0,29,29,0,0,0,0,-1,-1,-1,-1,0,0,0
2,2018-12-10-11:59:00--2018-12-10-12:05:04,3,0.008241758241758242,0.06593406593406594,0.0027472527472527475,0.0,29,29,0,0,0,0.4,30,16,59,144,0,0,0


In [53]:
def filter(inputFile, outputFile):
    """
    Only Storing those intervals roughly within 5 mins
    """
    import time
    import datetime
    def unixTime_intervals(element):
        print(element)
        fromTime, toTime = element[:40].split("--")
        # Conver to unix time
        fromTime = time.mktime(datetime.datetime.strptime(fromTime, "%Y-%m-%d-%H:%M:%S").timetuple()) 
        toTime = time.mktime(datetime.datetime.strptime(toTime, "%Y-%m-%d-%H:%M:%S").timetuple()) 
        return fromTime, toTime
    
    inputFile = [line.strip() for line in open("./"+inputFile, 'r')]
    f = open(outputFile, "a+")
    f.write(inputFile[0]+"\n")
    f.close() 
    for line in inputFile[1:]:
        fromTime, toTime = unixTime_intervals(line)
        if (toTime - fromTime) >= 300 and (toTime - fromTime) <= 305:
            f = open(outputFile, "a+")
            f.write(line+"\n")
            f.close() 
    return

filter("features_intermediate.csv", "final_fetures_file.csv")

2018-12-10-11:58:31--2018-12-10-11:59:00,0,0.0,0.0,0.0,0.0,29,29,0,0,0,0,-1,-1,-1,-1,0,0,0
2018-12-10-11:59:00--2018-12-10-12:05:04,3,0.008241758241758242,0.06593406593406594,0.0027472527472527475,0.0,29,29,0,0,0,0.4,30,16,59,144,0,0,0
